

- cf. [Google Colab で LangGraph を試す](https://note.com/npaka/n/n053a3cb78311?sub_rt=share_h)

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

assert "OPENAI_API_KEY" in os.environ
assert "TAVILY_API_KEY" in os.environ

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

In [3]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, streaming=True)

In [5]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

In [6]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


workflow = StateGraph(AgentState)

In [7]:
import json
from langgraph.prebuilt import ToolInvocation
from langchain_core.messages import FunctionMessage


# agentノードの次に呼び出されるノードを決定する関数の準備
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]

    # function_call がない場合は終了
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # そうでない場合は続行
    else:
        return "continue"


# agentノードの処理を定義する関数の準備
def call_model(state):
    messages = state["messages"]

    # モデルの呼び出し
    response = model.invoke(messages)

    # メッセージリストに追加するメッセージを返す
    return {"messages": [response]}


# actionノードの処理を定義する関数の準備
def call_tool(state):
    messages = state["messages"]
    last_message = messages[-1]

    # ツール呼び出しのアクションを生成
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(
            last_message.additional_kwargs["function_call"]["arguments"]
        ),
    )
    print(f"[DEBUG] {action=}")
    # アクション実行
    response = tool_executor.invoke(action)

    # 関数メッセージのインスタンスを生成
    function_message = FunctionMessage(content=str(response), name=action.tool)

    # メッセージリストに追加するメッセージを返す
    return {"messages": [function_message]}

In [8]:
from langgraph.graph import END

# グラフに2つのノードを追加
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# agentをエントリポイントとして指定
workflow.set_entry_point("agent")

# agentノードに条件付きエッジを追加
workflow.add_conditional_edges(
    "agent",  # 開始ノード
    should_continue,  # 呼び出されるノードを決定する関数
    {"continue": "action", "end": END},  # actionノードに遷移  # 終了
)

# actionノードからagentノードへのノーマルエッジを追加
workflow.add_edge("action", "agent")

# コンパイル
app = workflow.compile()

In [16]:
from langchain_core.messages.ai import AIMessage
from langchain_core.messages.function import FunctionMessage
from langchain_core.messages.human import HumanMessage


def print_response(response: dict):
    for itm in response["messages"]:
        if isinstance(itm, HumanMessage):
            print("Human:", itm.content)
        elif isinstance(itm, FunctionMessage):
            print("Function:", itm.content)
        elif isinstance(itm, AIMessage):
            print("AI:", itm.content)
    return

In [17]:
from langchain_core.messages import HumanMessage

# 質問応答
inputs = {"messages": [HumanMessage(content="日本の首都は？")]}
response = app.invoke(inputs)
print_response(response)

Human: 日本の首都は？
AI: 東京です。


In [18]:
from langchain_core.messages import HumanMessage

# 質問応答
inputs = {"messages": [HumanMessage(content="日本で一番高い山の名前と標高は？")]}
response = app.invoke(inputs)
print_response(response)

[DEBUG] action=ToolInvocation(tool='tavily_search_results_json', tool_input={'query': '日本で一番高い山の名前と標高'})
Human: 日本で一番高い山の名前と標高は？
AI: 
Function: [{'url': 'https://tabi-mag.jp/mountain-top10/', 'content': 'ランキング 2021.05.14 2023.04.20プレスマンユニオン編集部 日本の山 標高ベスト10  富士山 剣ヶ峰 日本の最高峰、富士山の最高地点が標高3776mの剣ヶ峰。かつてはレーダードームがあり、新幹線の車窓などからも視認できましたが、現在は富士山特別地域気象観測所が設置されています。 2位 北岳 3193m\u3000（南アルプス）  日本のマッターホルンともいわれる北アルプスのシンボル的な存在、槍ヶ岳（やりがたけ）。標高3180mで、日本第5位の高峰。長野県松本市、大町市、岐阜県高山市の境にそびえ、燕岳（つばくろだけ）と結ぶコースが、北アルプス初心者にも人気の表銀座で  日本の山標高ベスト10は、独立峰の富士山を1位に、2位〜10位には南アルプスが、日本第二の高峰・北岳など4座、北アルプスが3位の奥穂高岳など5座がランクイン。白峰三山、悪沢岳（荒川東岳）・赤石岳、槍穂高連峰と、いずれも日本アルプスの人気の日本の山 標高ベスト10 シェア ツイート 日本の山標高ベスト10は、独立峰の富士山を1位に、2位〜10位には南アルプスが、日本第二の高峰・北岳など4座、北アルプスが3位の奥穂高岳など5座がランクイン。 白峰三山、悪沢岳（荒川東岳）・赤石岳、槍穂高連峰と、いずれも日本アルプスの人気の縦走路で氷河地形がアルペンムードを高めています。 1位 富士山 3776m 名称 ：富士山・剣ヶ峰 山域 ：富士山（独立峰） 場所 ：静岡県富士宮市 三角点名など ：測定点／二等三角点は3775.63m 富士山 剣ヶ峰 日本の最高峰、富士山の最高地点が標高3776mの剣ヶ峰。 かつてはレーダードームがあり、新幹線の車窓などからも視認できましたが、現在は富士山特別地域気象観測所が設置されています。'}]
AI: 日本で一番高い山は富士山です。富士山の最高地点の標高は377

In [20]:
# ストリーミング （処理ステップ毎）
inputs = {"messages": [HumanMessage(content="日本で一番高い山の名前と標高を調べた上で、その山の歴史について説明してください")]}
for output in app.stream(inputs):
    for key, tkn in output.items():
        print(f"Output from node '{key}': ", end="")
        for msg in tkn["messages"]:
            print(msg.content.replace("\n\n", "\n\t"))
    print("\n---\n")

Output from node 'agent': 

---

[DEBUG] action=ToolInvocation(tool='tavily_search_results_json', tool_input={'query': '日本で一番高い山の名前と標高'})
Output from node 'action': [{'url': 'https://tabi-mag.jp/mountain-top10/', 'content': 'ランキング 2021.05.14 2023.04.20プレスマンユニオン編集部 日本の山 標高ベスト10  富士山 剣ヶ峰 日本の最高峰、富士山の最高地点が標高3776mの剣ヶ峰。かつてはレーダードームがあり、新幹線の車窓などからも視認できましたが、現在は富士山特別地域気象観測所が設置されています。 2位 北岳 3193m\u3000（南アルプス）  日本のマッターホルンともいわれる北アルプスのシンボル的な存在、槍ヶ岳（やりがたけ）。標高3180mで、日本第5位の高峰。長野県松本市、大町市、岐阜県高山市の境にそびえ、燕岳（つばくろだけ）と結ぶコースが、北アルプス初心者にも人気の表銀座で  日本の山標高ベスト10は、独立峰の富士山を1位に、2位〜10位には南アルプスが、日本第二の高峰・北岳など4座、北アルプスが3位の奥穂高岳など5座がランクイン。白峰三山、悪沢岳（荒川東岳）・赤石岳、槍穂高連峰と、いずれも日本アルプスの人気の日本の山標高ベスト10は、独立峰の富士山を1位に、2位〜10位には南アルプスが、日本第二の高峰・北岳など4座、北アルプスが3位の奥穂高岳など5座がランクイン。白峰三山、悪沢岳（荒川東岳）・赤石岳、槍穂高連峰と、いずれも日本アルプスの人気の縦走路で氷河地形がアルペンムードを高め ...'}]

---

Output from node 'agent': 日本で一番高い山は富士山です。その最高地点は剣ヶ峰で、標高は3776メートルです。
	富士山は日本のシンボルとして知られており、世界遺産にも登録されています。古くから信仰の対象とされ、多くの登山者や観光客が訪れます。
	富士山の歴史は古く、約10万年前に形成されたと考えられています。火山活動によって形成されたこの山は、その美しい姿勢と独特の形状で多くの人々を

In [33]:
for msg in output["__end__"]["messages"]:
    # if hasattr(msg, "name"):
    #     print(msg.name)
    print(msg)

content='日本で一番高い山の名前と標高を調べた上で、その山の歴史について説明してください'
content='' additional_kwargs={'function_call': {'arguments': '{\n  "query": "日本で一番高い山の名前と標高"\n}', 'name': 'tavily_search_results_json'}}
content="[{'url': 'https://tabi-mag.jp/mountain-top10/', 'content': 'ランキング 2021.05.14 2023.04.20プレスマンユニオン編集部 日本の山 標高ベスト10  富士山 剣ヶ峰 日本の最高峰、富士山の最高地点が標高3776mの剣ヶ峰。かつてはレーダードームがあり、新幹線の車窓などからも視認できましたが、現在は富士山特別地域気象観測所が設置されています。 2位 北岳 3193m\\u3000（南アルプス）  日本のマッターホルンともいわれる北アルプスのシンボル的な存在、槍ヶ岳（やりがたけ）。標高3180mで、日本第5位の高峰。長野県松本市、大町市、岐阜県高山市の境にそびえ、燕岳（つばくろだけ）と結ぶコースが、北アルプス初心者にも人気の表銀座で  日本の山標高ベスト10は、独立峰の富士山を1位に、2位〜10位には南アルプスが、日本第二の高峰・北岳など4座、北アルプスが3位の奥穂高岳など5座がランクイン。白峰三山、悪沢岳（荒川東岳）・赤石岳、槍穂高連峰と、いずれも日本アルプスの人気の日本の山標高ベスト10は、独立峰の富士山を1位に、2位〜10位には南アルプスが、日本第二の高峰・北岳など4座、北アルプスが3位の奥穂高岳など5座がランクイン。白峰三山、悪沢岳（荒川東岳）・赤石岳、槍穂高連峰と、いずれも日本アルプスの人気の縦走路で氷河地形がアルペンムードを高め ...'}]" name='tavily_search_results_json'
content='日本で一番高い山は富士山です。その最高地点は剣ヶ峰で、標高は3776メートルです。\n\n富士山は日本のシンボルとして知られており、世界遺産にも登録されています。古くから信仰の対象とされ、多くの登山者や観光客が訪れます。\n\n富士山の歴史は古く、約10万年前に形成された